<a href="https://colab.research.google.com/github/rajni-arora/Deep-Learning-Projects/blob/main/Bert_tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stage 1: Importing dependencies

In [1]:
!wget http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip

--2021-10-26 14:43:10--  http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip [following]
--2021-10-26 14:43:11--  https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81363704 (78M) [application/zip]
Saving to: ‘trainingandtestdata.zip’

trainingandtestdata 100%[===================>]  77.59M  3.91MB/s    in 16s     

2021-10-26 14:43:27 (4.98 MB/s) - ‘trainingandtestdata.zip’ saved [81363704/81363704]



In [2]:
!unzip '/content/trainingandtestdata.zip' 

Archive:  /content/trainingandtestdata.zip
  inflating: testdata.manual.2009.06.14.csv  
  inflating: training.1600000.processed.noemoticon.csv  


In [3]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random

from google.colab import drive

In [4]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 41 kB 154 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30534 sha256=7ec25be7c5d1af9597931abeb0d17e55e7384a567a70d219b10c694e3d93df7b
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19473 sha256=02f349538aeb8e32d4af4589e95a30abb54001c991b282e9c48518c07eda4755
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7912 sha256=35d2f2a5a44aa18f18b69f2100c0d32f3d0b224b9a5f818613fcf3e5f62d8f6a
  Stored in directory: /root/.cache/pip/wheels/e1/11/67/33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf9969716
Successfully built bert-for-tf2 params-flow py-params
     |████████████████████████████████| 1.2 MB 13.0 MB/s 


In [5]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

# Stage 2: Data preprocessing

## Loading files

We import files from our personal Google drive.

In [ ]:
drive.mount("/content/drive")

In [8]:
cols = ["sentiment", "id", "date", "query", "user", "text"]
data = pd.read_csv(
    '/content/training.1600000.processed.noemoticon.csv',
    header=None,
    names=cols,
    engine="python",
    encoding="latin1")

In [9]:
data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

In [10]:
data.head(5)

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


## Preprocessing

### Cleaning

In [11]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    # Delete the @
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    # Delete URL links
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    # Just keep letters and important punctuation
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    # Remove additional spaces
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [12]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [13]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

### Tokenization

We need to create a BERT layer to have access to meta data for the tokenizer (like vocab size).

In [14]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [15]:
def encode_sentence(sent):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent))

In [16]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

### Dataset creation

We will create padded batches (so we pad sentences for each batch inpedendently), this way we add the minimum of padding tokens possible. For that, we sort sentences by length, apply padded_batches and then shuffle.

In [17]:
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_inputs)]
random.shuffle(data_with_len)
data_with_len.sort(key=lambda x: x[2])
sorted_all = [(sent_lab[0], sent_lab[1])
              for sent_lab in data_with_len if sent_lab[2] > 7]

In [18]:
# A list is a type of iterator so it can be used as generator for a dataset
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [19]:
next(iter(all_dataset))

(<tf.Tensor: shape=(8,), dtype=int32, numpy=
 array([ 8038,  2100,   999,  2009,  1005,  1055, 24057,   999],
       dtype=int32)>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)

In [20]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [21]:
next(iter(all_batched))

(<tf.Tensor: shape=(32, 8), dtype=int32, numpy=
 array([[ 8038,  2100,   999,  2009,  1005,  1055, 24057,   999],
        [ 2145,  2007, 26324,  2129,  2015,  2055,  1057,  1029],
        [ 1051,  8913,  2099,  1045,  1005,  1049,  3374,   999],
        [ 1045,  3335,  2026,  6898,  1012,  1012,  2272,  2188],
        [14475,  2026, 11360,  2000,  1037,  2367,  5353,  1012],
        [ 2026,  2608,  2024,  2061,  5296,  2013,  1996,  3103],
        [26114,  2978, 24026,  2000,  2131,  1996,  6681,  2648],
        [ 2593,  2126,  1045,  2572,  2893,  1037, 12191,  3892],
        [ 2038,  2572,  7610,  8040,  9072,  2006, 28401,  1012],
        [ 5409,  2954,  3496,  2412,  1044,  1029,  1044,  1029],
        [ 4931,   999,   999,   999,  2619, 10312,  1996,  3103],
        [ 2718,  1996,  5742,  4770,  2175,  2000,  1037, 12403],
        [19340, 19340,  1012,  1012,  1012,  1012,  2061,  5305],
        [ 2183,  2000, 24531,  1045,  2145,  5223,  2008,  2465],
        [ 1045,  5993,  5959

In [22]:
NB_BATCHES = math.ceil(len(sorted_all) / BATCH_SIZE)
NB_BATCHES_TEST = NB_BATCHES // 10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

# Stage 3: Model building

In [23]:
class DCNN(tf.keras.Model):
    
    def __init__(self,
                 vocab_size,
                 emb_dim=128,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="dcnn"):
        super(DCNN, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocab_size,
                                          emb_dim)
        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x) # (batch_size, nb_filters, seq_len-1)
        x_1 = self.pool(x_1) # (batch_size, nb_filters)
        x_2 = self.trigram(x) # (batch_size, nb_filters, seq_len-2)
        x_2 = self.pool(x_2) # (batch_size, nb_filters)
        x_3 = self.fourgram(x) # (batch_size, nb_filters, seq_len-3)
        x_3 = self.pool(x_3) # (batch_size, nb_filters)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

# Stage 4: Training

In [24]:
VOCAB_SIZE = len(tokenizer.vocab)
EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [25]:
Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [26]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [28]:
checkpoint_path = '/content/ckpt_bert_tok/'

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest Checkpoint restored!")

In [29]:
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint saved at {}.".format(checkpoint_path))

In [30]:
Dcnn.fit(train_dataset,
         epochs=NB_EPOCHS,
         callbacks=[MyCustomCallback()])

Epoch 1/5
37196/37196 [==============================] - 1931s 52ms/step - loss: 0.4291 - accuracy: 0.8026
Checkpoint saved at /content/ckpt_bert_tok/.
Epoch 2/5
37196/37196 [==============================] - 1938s 52ms/step - loss: 0.3815 - accuracy: 0.8302
Checkpoint saved at /content/ckpt_bert_tok/.
Epoch 3/5
37196/37196 [==============================] - 1933s 52ms/step - loss: 0.3419 - accuracy: 0.8514
Checkpoint saved at /content/ckpt_bert_tok/.
Epoch 4/5
37196/37196 [==============================] - 1934s 52ms/step - loss: 0.3014 - accuracy: 0.8714
Checkpoint saved at /content/ckpt_bert_tok/.
Epoch 5/5
37196/37196 [==============================] - 1917s 51ms/step - loss: 0.2634 - accuracy: 0.8889
Checkpoint saved at /content/ckpt_bert_tok/.


# Stage 5: Evaluation

In [31]:
results = Dcnn.evaluate(test_dataset)
print(results)

4132/4132 [==============================] - 26s 6ms/step - loss: 0.4276 - accuracy: 0.8262
[0.42759209871292114, 0.8261737823486328]


In [32]:
def get_prediction(sentence):
    tokens = encode_sentence(sentence)
    inputs = tf.expand_dims(tokens, 0)

    output = Dcnn(inputs, training=False)

    sentiment = math.floor(output*2)

    if sentiment == 0:
        print("Output of the model: {}\nPredicted sentiment: negative.".format(
            output))
    elif sentiment == 1:
        print("Output of the model: {}\nPredicted sentiment: positive.".format(
            output))

In [33]:
get_prediction("This movie was pretty interesting.")

Output of the model: [[0.930081]]
Predicted sentiment: positive.


In [34]:
get_prediction("I'd rather not do that again.")

Output of the model: [[0.10450524]]
Predicted sentiment: negative.
